In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Function to fetch weather data from the Weather API
def fetch_weather_data(api_key, location, days=30):
    # Base URL for the weather API
    url = f"http://api.weatherapi.com/v1/history.json?key={api_key}&q={location}&dt="
    weather_data = []

    # Get the last 'days' days of weather data
    for i in range(days):
        date = (datetime.now() - timedelta(days=i)).strftime("%Y-%m-%d")
        response = requests.get(url + date)

        if response.status_code == 200:
            data = response.json()
            try:
                # Extract relevant information
                daily_data = {
                    "date": date,
                    "sunlight_hours": data['forecast']['forecastday'][0]['day'].get('sunshine', 0),  # Default to 0 if not found
                    "cloud_cover": data['forecast']['forecastday'][0]['day'].get('cloud', 0),        # Default to 0 if not found
                    "temperature": data['forecast']['forecastday'][0]['day'].get('avgtemp_c', 0),    # Default to 0 if not found
                    "solar_energy_production": None  # Placeholder for actual production data
                }
                weather_data.append(daily_data)
            except KeyError as e:
                print(f"Key error: {e} on date: {date}")
                print("Response data:", data)  # Print the response to understand its structure
        else:
            print(f"Error fetching data for {date}: {response.status_code}")

    return pd.DataFrame(weather_data)

# Function to create synthetic solar energy production data
def create_solar_energy_production(df):
    # Assume some coefficients for the model
    sunlight_factor = 1.5  # Solar energy production per sunlight hour
    temperature_factor = 0.1  # Additional energy per degree Celsius
    cloud_cover_penalty = -0.5  # Energy loss per percentage of cloud cover

    # Calculate solar energy production
    df['solar_energy_production'] = (
        df['sunlight_hours'] * sunlight_factor +
        df['temperature'] * temperature_factor +
        df['cloud_cover'] * cloud_cover_penalty
    ).clip(lower=0)  # Ensure no negative production values

    return df

# Replace with your actual API key and desired location
API_KEY = "d224a9f66ffa425eab3180904242310"  # Add your Weather API key here
LOCATION = "Nagpur"

# Fetch weather data
weather_df = fetch_weather_data(API_KEY, LOCATION)

# Create synthetic solar energy production data
weather_df = create_solar_energy_production(weather_df)

# Check for missing values and fill them if necessary
weather_df.ffill(inplace=True)  # Use ffill to fill missing values

# Prepare data for training
X = weather_df[['sunlight_hours', 'cloud_cover', 'temperature']]
y = weather_df['solar_energy_production']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse:.2f}')
print(f'R^2 Score: {r2:.2f}')

# Function to predict solar energy production for a new day
def predict_solar_energy(model, sunlight_hours, cloud_cover, temperature):
    input_data = pd.DataFrame([[sunlight_hours, cloud_cover, temperature]], columns=['sunlight_hours', 'cloud_cover', 'temperature'])
    predicted_production = model.predict(input_data)
    return predicted_production[0]

# Example prediction
new_sunlight_hours = 10  # Example sunlight hours
new_cloud_cover = 20     # Example cloud cover percentage
new_temperature = 25      # Example temperature in Celsius

predicted_energy = predict_solar_energy(model, new_sunlight_hours, new_cloud_cover, new_temperature)
print(f'Predicted Solar Energy Production: {predicted_energy:.2f} kWh')


Mean Squared Error: 0.00
R^2 Score: 0.97
Predicted Solar Energy Production: 2.52 kWh
